- True implementation
  - LightGBM (scikit learn)
  - Loop: attack & train
    - annotate full-attacks dataset with costs
    - generate attacked datasets
    - continue training from previous forest with new data

In [8]:
import os
import pickle
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm
#import functools
#from os import listdir
#from os.path import isfile, join

In [10]:
DATASETS_PATH = "../data/census"
MODELS_PATH = "../out/models"
ATTACKER = "strong" # weak
TRAINING_SET="train_ori.csv.bz2" # original training set
TRAINING_SET_ATT="train_"+ATTACKER+"_att.csv.bz2" # perturbed training set
VALIDATION_SET="valid_ori.csv.bz2" # original validation set
VALIDATION_SET_ATT="valid_"+ATTACKER+"_att.csv.bz2" # perturbed validation set
TEST_SET="test_ori.csv.bz2" # original test set
TEST_SET_ATT="test_"+ATTACKER+"_att.csv.bz2" # perturbed test set

In [11]:
def load_dataset(path, dataset_filename, sep=","):
    return pd.read_csv(path+"/"+dataset_filename, sep=sep)

In [13]:
def infer_categorical_features(dataset):
    categorical_features = []
    for column in dataset.columns:
        if dataset[column].dtype == 'object':
            categorical_features.append(column)
    return categorical_features
            
def label_encode(dataset, categorical_features):
    dataset_le = dataset.copy()
    for column in dataset_le.columns:
        if column in categorical_features:
            dataset_le[column] = dataset_le[column].astype('category')
            dataset_le[column] = dataset_le[column].cat.codes
    return dataset_le

## Prepare Data

In [12]:
TRAIN = load_dataset(DATASETS_PATH, TRAINING_SET)
TRAIN_ATT = load_dataset(DATASETS_PATH, TRAINING_SET_ATT)

VALID = load_dataset(DATASETS_PATH, VALIDATION_SET)
VALID_ATT = load_dataset(DATASETS_PATH, VALIDATION_SET_ATT)

TEST = load_dataset(DATASETS_PATH, TEST_SET)
TEST_ATT = load_dataset(DATASETS_PATH, TEST_SET_ATT)

In [14]:
TRAIN_ATT_OFFSETS = TRAIN_ATT['instance_id'].value_counts().sort_index().values
VALID_ATT_OFFSETS = VALID_ATT['instance_id'].value_counts().sort_index().values
TEST_ATT_OFFSETS = TEST_ATT['instance_id'].value_counts().sort_index().values

In [15]:
def process_categorical_features(dataset):
    fx = infer_categorical_features(dataset)
    print("List of categorical features: [{}]"
          .format(", ".join([cf for cf in fx])))
    return label_encode(dataset, set(fx))

TRAIN = process_categorical_features(TRAIN)
TRAIN_ATT = process_categorical_features(TRAIN_ATT.iloc[:,1:])

VALID = process_categorical_features(VALID)
VALID_ATT = process_categorical_features(VALID_ATT.iloc[:,1:])

TEST = process_categorical_features(TEST)
TEST_ATT = process_categorical_features(TEST_ATT.iloc[:,1:])

List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]
List of categorical features: [workclass, education, marital_status, occupation, relationship, race, sex, native_country]


## Load some model

In [18]:
def save_model(model_filename, model):
    with open(model_filename, 'wb') as fout:
        pickle.dump(model, fout)

In [19]:
def load_model(model_filename):
    with open(model_filename, 'rb') as fin:
        return pickle.load(fin)

In [23]:
std_model = load_model("../out/models/std_strong_200.pkl")

## Adversarial Boosting

In [97]:
def AdvBoosting_gen_data(model, data, groups):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    groups : grouping of same attacked instance 
    returns the new data matrix and new groups
    
    WARNING: currently works only for binary classification
    '''
    # score the datataset
    labels = data.iloc[:,-1]
    
    predictions = model.predict(data.iloc[:,:-1]) # exclude labels
    # binarize
    predictions = (predictions>0).astype(np.float)
    predictions = 2*predictions - 1
    
    # check mispredictions
    matchings = labels * predictions
    
    # select original data + attacked instances
    new_selected = [] # id of selected instances
    new_groups   = []
    
    offset = 0
    for g in groups:
        if g==1:
            # there are not attacks, just add original
            new_selected += [offset]
            new_groups   += [1]
        else:
            # get a slice of the matching scores
            g_matchings = matchings[offset:offset+g].values

            # most misclassified (smallest margin)
            # skip original
            adv_instance = np.argmin(g_matchings[1:])+1

            # add original and adversarial
            new_selected += [offset, adv_instance]
            new_groups   += [2]
        
        offset += g
    
    new_dataset = data.iloc[new_selected,:]
    
    return new_dataset, new_groups

In [181]:
# Our custom metric


def binary_log_loss(pred, true_label):

    return np.log(1.0 + np.exp(-pred * true_label))

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss(preds, train_data):
    
    labels = train_data.get_label()
    losses = np.log(1.0 + np.exp(-preds*labels))
    avg_loss = np.mean(losses)
    
    return 'avg_binary_log_loss', avg_loss, False

# self-defined eval metric
# f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
def avg_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    offset = 0
    max_logloss = []
    avg_max_logloss = 0.0
    
    if attack_lens is not None:
    
        for atk in attack_lens:
            losses = [binary_log_loss(h,t) for h,t in zip(preds[offset:offset+atk], labels[offset:offset+atk])]
            max_logloss.append(max(losses))

            offset += atk
        
        avg_max_logloss = np.mean(max_logloss)  

    return 'avg_binary_log_loss_under_max_attack', avg_max_logloss, False

def avg_non_interferent_log_loss(preds, train_data, alpha=1.0):
    
    # binary logloss under maximal attack
    _, loss_uma, _    = avg_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    _, loss_plain, _  = avg_log_loss(preds, train_data)
    
    # combine the above two losses together
    weighted_loss = alpha*loss_uma + (1.0-alpha)*loss_plain

    return 'avg_non_interferent_log_loss [alpha={}]'.format(alpha), weighted_loss, False

def optimize_log_loss_uma(preds, train_data):
    labels = train_data.get_label()
    attack_lens = train_data.get_group()
    
    grads = np.zeros_like(labels, dtype=np.float64)
    hess = np.zeros_like(grads)
    
    if attack_lens is not None:

        norm = 1.0 / float(len(attack_lens))

        offset = 0
        for atk in attack_lens:
            exp_pl = np.exp(- preds[offset:offset+atk] * labels[offset:offset+atk])

            inv_sum = 1.0 / np.sum(1.0 + exp_pl)

            x_grad = inv_sum * exp_pl

            grads[offset:offset+atk] = norm * x_grad * (- labels[offset:offset+atk])
            hess[offset:offset+atk]  = norm * x_grad * (1.0 - x_grad)

            offset += atk    
    
    return grads, hess

def optimize_non_interferent_log_loss(preds, train_data, alpha=1.0):
    # binary logloss under maximal attack
    grads_uma, hess_uma = optimize_log_loss_uma(preds, train_data)
    
    # binary logloss (plain)
    grads_plain, hess_plain = optimize_log_loss(preds, train_data)
    
    # combine the above two losses together
    grads = alpha*grads_uma + (1.0-alpha)*grads_plain
    hess  = alpha*hess_uma  + (1.0-alpha)*hess_plain
    
    return grads, hess

def optimize_log_loss(preds, train_data):
    labels = train_data.get_label()
    exp_pl = np.exp(preds * labels)
    # http://www.wolframalpha.com/input/?i=differentiate+log(1+%2B+exp(-kx)+)
    grads = -labels / (1.0 +  exp_pl)  
    # http://www.wolframalpha.com/input/?i=d%5E2%2Fdx%5E2+log(1+%2B+exp(-kx)+)
    hess = labels**2 * exp_pl / (1.0 + exp_pl)**2 

    # this is to optimize average logloss
    norm = 1.0/len(preds)
    grads *= norm
    hess *= norm
    
    return grads, hess




def AdvBoosting_extend_model(data, input_model=None, num_trees=1, params=None):
    ''' 
    model  : is the LightGBM Model
    data   : data matrix with all valid attacks (last column is label)
    returns the new model (is model modified inplace?)
    '''
    
    if params is None:
#         params = {
#             #    "max_bin": 511,
#             "learning_rate": 0.05,
#             "boosting_type": "gbdt",#"rf"
#             "objective": "regression_l2", #"binary",
#             #"metric": ["None"], # We use our own implementation of binary log loss (i.e., optimize_log_loss) 
#             #                    # instead of the default one (i.e., "binary_logloss"), which may be in fact cross-entropy
#             "num_leaves": 16, # 15
#             "verbose": 1,
#             "min_data_in_leaf": 20,
#             # "bagging_freq": 1,
#             "bagging_fraction": 1.0,
#             "feature_fraction": 0.0,
#             "boost_from_average": True
#         }
        params = {
            'task':'train',
            'learning_rate': 0.1,
            'num_leaves': 16,
            'min_data_in_leaf': 20, #[1, 20]
            'metirc': ['l2'],
            'verbose': 1
        }  

    lgbm_train = lightgbm.Dataset(data=data.iloc[:,:-1].values , 
                                  label=data.iloc[:,-1].values )
    
    print (params)

    lgbm_info = {}
    lgbm_model = lightgbm.train(params, lgbm_train, 
                                num_boost_round = num_trees, 
                                #init_model = input_model,
#                                 fobj = optimize_log_loss, 
#                                 feval = avg_log_loss,
                                evals_result = lgbm_info,
                                verbose_eval=1)

#     lgbm_model = lightgbm.train(params=params, 
#                                 train_set=lgbm_train, 
#                                 num_boost_round=num_trees, 
#                                 evals_result = lgbm_info,
#                                 fobj = fobj,
#                                 feval = feval,
#                                 early_stopping_rounds=50,
#                                 verbose_eval=20
#                                )

    print (lgbm_info)

    return lgbm_model, lgbm_info

In [138]:
adv_data, _ = AdvBoosting_gen_data(std_model, TRAIN_ATT, TRAIN_ATT_OFFSETS)

In [121]:
next_model, next_model_info = AdvBoosting_extend_model(adv_data, num_trees=1000)

print (next_model_info)

{}
{}


In [156]:
std_model.best_iteration

104

In [180]:
base_model, base_model_info = AdvBoosting_extend_model(adv_data, num_trees=100)

{'task': 'train', 'learning_rate': 0.1, 'num_leaves': 16, 'min_data_in_leaf': 20, 'metirc': ['l2'], 'verbose': 1}
{}
